In [3]:
import os
import cv2
from matplotlib import pyplot as plt
from matplotlib import patches
import numpy as np

In [4]:
TRAIN_PATH = os.getcwd() + '/Dataset/train'
TEST_PATH = os.getcwd() + '/Dataset/test'
CHANNEL1 = 0
CHANNEL2 = 1
CHANNEL3 = 2
H_MAX = 180
L_MAX = 256
S_MAX = 256
MASK = None
H_RANGE = [0, H_MAX]
L_RANGE = [0, L_MAX]
S_RANGE = [0, S_MAX]
# Fine tunning by hands!
TRAIN_SETTINGS = dict(dp=2, minDist=350, param1=650, param2=100, minRadius=175,
maxRadius=225)
# Fine tunning by hands!
TEST_SETTINGS = dict(dp=2, minDist=300, param1=500, param2=100, minRadius=130,
maxRadius=140)
GREEN_COLOR = 0, 1.0, 0
WHITE_COLOR = 1.0, 1.0, 1

In [5]:
def circles_check(path):
  filenames = os.listdir(path)
  for filename in filenames:
    image = plt.imread(path + '/' + filename)
    grayscale_image = cv2.imread(path + '/' + filename, cv2.IMREAD_GRAYSCALE)
# Fine tunning by hands!
    circles = cv2.HoughCircles(grayscale_image,
                              cv2.HOUGH_GRADIENT,
                              dp=2,
                              minDist=300,
                              param1=500,
                              param2=100,
                              minRadius=130,
                              maxRadius=140)
    ax = plt.subplot()
    if circles is not None:
      for circle in circles[0]:
        x, y, r = map(int, circle)
        draw_circle = patches.Circle((x, y),
                                    radius=r,
                                    color=GREEN_COLOR,
                                    linewidth=3,
                                    fill=False)
        ax.imshow(image)
        ax.add_patch(draw_circle)
    else:
      print('No circles found.')
      plt.show()

In [6]:
def get_circles(image, settings):
  intermediate_accumulator = np.zeros((H_MAX, L_MAX, S_MAX))
  circles = get_circles(image, TRAIN_SETTINGS)
  hls_image = cv2.cvtColor(image, cv2.COLOR_BGR2HLS)
  for circle in circles:
    x, y, r = map(int, circle)
    crop = hls_image[y-r:y+r, x-r:x+r, :]
    hist = cv2.calcHist([crop],
    [CHANNEL1, CHANNEL2, CHANNEL3],
    MASK,
    [H_MAX, L_MAX, S_MAX],
    H_RANGE + L_RANGE + S_RANGE,
    accumulate=False)
    intermediate_accumulator += hist
  return intermediate_accumulator

In [ ]:
def get_mean_hist(path, label):
  filenames = os.listdir(path + '/' + label)
  accumulator = np.zeros((H_MAX, L_MAX, S_MAX))
  for filename in filenames:
  image = cv2.imread(path + '/' + label + '/' + filename)
  intermediate_accumulator = get_intermediate_accumulator(image)
  accumulator += intermediate_accumulator
  mean_hist = accumulator / len(filenames)
  mean_hist = mean_hist.astype('float32')
  return mean_hist


IndentationError: ignored